### Import libs & db details

In [1]:
#import needed libraries
from sqlalchemy import create_engine
import pandas as pd
import os
import time
#get password from environmnet var
pwd = "postgres"
uid = "postgres"
server = "localhost"
db = "Adventureworks"
port = "5433"
#
engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:{port}/{db}')

### Read data

In [ ]:
df = pd.read_sql('Select * from humanresources.department', engine)
df = df[['departmentid', 'name', 'groupname', 'modifieddate']]
df

### Send data to topic

In [ ]:
from kafka import KafkaProducer, KafkaConsumer
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas import Timestamp

# Define Kafka server and topic
bootstrap_servers = ['localhost:29092']
topic_name = 'test2'

# Initialize producer
producer = KafkaProducer(bootstrap_servers=bootstrap_servers,
                         key_serializer=str.encode,
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

# Initialize consumer
consumer = KafkaConsumer(topic_name, bootstrap_servers=bootstrap_servers, group_id='youtube_tutorial')

# SQLAlchemy engine
pwd = "postgres"
uid = "postgres"
server = "localhost"
db = "Adventureworks"
port = "5433"
engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:{port}/{db}')

# Read data from the database
df = pd.read_sql('SELECT * FROM humanresources.department', engine)
df = df[['departmentid', 'name', 'groupname', 'modifieddate']]

# Send DataFrame rows to Kafka topic
for index, row in df.iterrows():
    department_data = row.to_dict()

    # Converter Timestamp para string antes de serializar para JSON
    for key, value in department_data.items():
        if isinstance(value, Timestamp):
            department_data[key] = value.strftime('%Y-%m-%d %H:%M:%S')

    department_id = department_data['departmentid']

    # Produce the message to the Kafka topic
    producer.send(topic_name, key=str(department_id), value=department_data)

    print(f"Row Inserted: {department_data}")

# Close the producer to ensure all messages are sent
producer.close()

